# Scripts for the Analysis of RNASeq data

This is a work in progress.

github: rodriguezmDNA 

How to use this pipeline?

There are some scripts for the analysis of RNA seq data. In the future I will add some other functionalities and wrappers for other mapping software (bwa, STAR).


In [1]:
ls Scripts

01a_QCraw.sh
01b_readTrim.sh
01c_QCtrim.sh
02a.GetLibrarySizes.sh
02b.plotLibrarySizes.R
03a_makeBowtieIndex.sh
03b_bowtieMapping.sh
03c_BowtieAlignmentStats.R
04a_annotationHTSeq.sh
04b_countsFromcDNAMapping.sh
04c_makeItCount.R
04d_GenomicAlignments_count_genomeMapReads.R
GOSeq.R
RNASeq_Analysis_Pipeline.ipynb
functions_jrm.R
metaFunctions_forNetworkAnalysis.R
wip_02c_DeduplicatedPercentage.sh
zCodeHere.sh


You need to have the following programs installed in your path. 

* fastqc - For quality control.
* fastx_trimmer - For trimming barcodes.
* reaper - For cleaning reads.
* bowtie or bowtie2 - For alignment.
* htseq-count - For annotating reads (when mapping to a reference genome).


### Folder Organization

The scripts are pretty flexible but I left some options hardcoded (like the names of the input and output directories). You can modify these at your own risk.

__Recommended organization__

* `Scripts/` - Where the scripts 
* `RawData/` - Where the sequences live (with either `.fq` or `.fastq` suffix). The scripts assume data is gzipped. 
    > `RawData/file.f*q.gz`

As you run the scripts folders will be created. A `log` folder is created for each and the output plus other information is recorded in log files. Some of them are necessary for stats about the analysis (like percentage of uniquely aligned reads).

### Quality Control (before Processing)

Run the `01a_QCraw.sh`. This will create the `00_QC` directory and the `a_RawQC` subfolder with the results of quality  analysis on raw reads.

fastqc will create html files with the summary on the quality of the reads. I like to do the analysis before and after trimming to compare the outputs and in case there's some contaminant (adapter sequence) I'm missing I can include it on the trimming script.

In [51]:
Scripts/01a_QCraw.sh



Started on Fri Oct  6 11:17:44 PDT 2017

/Users/jrm/Desktop/deleteThis/Scripts/../RawData/test1.fq.gz
Started analysis of test1.fq.gz
Approx 10% complete for test1.fq.gz
Approx 20% complete for test1.fq.gz
Approx 30% complete for test1.fq.gz
Approx 40% complete for test1.fq.gz
Approx 50% complete for test1.fq.gz
Approx 60% complete for test1.fq.gz
Approx 70% complete for test1.fq.gz
Approx 80% complete for test1.fq.gz
Approx 90% complete for test1.fq.gz
Approx 100% complete for test1.fq.gz
/Users/jrm/Desktop/deleteThis/Scripts/../RawData/test2.fq.gz
Started analysis of test2.fq.gz
Approx 10% complete for test2.fq.gz
Approx 20% complete for test2.fq.gz
Approx 30% complete for test2.fq.gz
Approx 40% complete for test2.fq.gz
Approx 50% complete for test2.fq.gz
Approx 60% complete for test2.fq.gz
Approx 70% complete for test2.fq.gz
Approx 80% complete for test2.fq.gz
Approx 90% complete for test2.fq.gz
Approx 100% complete for test2.fq.gz

Parameters used: fastqc file --extract -o ToDir


### Trim Reads 

Use `01b_readTrim.sh -h` to get help on the script.

This has two modes, just trim barcodes (with fastx_trimmer) or couple with reaper to trim and remove low qual parts and adapter contamination. 

Check more possible adapters here: https://support.illumina.com/content/dam/illumina-support/documents/documentation/chemistry_documentation/experiment-design/illumina-adapter-sequences-1000000002694-02.pdf

In [25]:
# Get help:
Scripts/01b_readTrim.sh -h | head

Wrapper to trim and clean RNASeq data.
Usage: Scripts/01b_readTrim.sh [options...] 



   -h            this useful help
   -m            mode: A) Barcode removal (First n bp)
                       B) Barcode & trim (Uses reaper)
   -n           length of barcode (removes first n bases).
                ie -n 9 will remove the first 8bp.
                Default value: 1; Use 8 for processing BRAD-Seq data
   -t           tabu sequence - See reaper documentation 


In [52]:
# Run
Scripts/01b_readTrim.sh -m B -n 10 -p meta/reaperArguments.txt


Reading meta/reaperArguments.txt


Started on Fri Oct  6 11:22:24 PDT 2017
Trimming option B
C. Remove barcodes and adapters. Trim reads based on quality and complexity [fastx & reaper]

Trimming test1.fq.gz
B. Remove barcodes and adapters. Trim reads based on quality and complexity [fastx & reaper]
Using GATCGGAAGAGCACACGTCTGAACTCCAGTCAC as tabu sequences
Using ATCTCGTATGCCGTCTTCTGCTTG as adapter sequences
adapter ATCTCGTATGCCGTCTTC
---
mRpm   million reads per minute
mNpm   million nucleotides per minute
mCps   million alignment cells per second
lint   total removed reads (per 10K), sum of columns to the left
25K reads per dot, 1M reads per line  seconds  mr mRpm mNpm mCps {error qc  low  len  NNN tabu nobc cflr  cfl lint   OK} per 10K
Trimming: base 10 to 0
Input: 10000 reads.
Output: 10000 reads.

[reaper] check 0 errors, 0 reads truncated, 9291 clean, 709 lint, 10000 total

 ----------------------------------------------------------
Trimming test2.fq.gz
B. Remove barcodes and ada

### Quality Control (after trimming)

Run the `01c_QCtrim.sh`. This will create the `00_QC` directory (if it doesn't exists already) and the `b_trimQC` subfolder with the results of quality analysis on trimmed reads.

In [53]:
Scripts/01c_QCtrim.sh


Started on Fri Oct  6 11:22:36 PDT 2017

/Users/jrm/Desktop/deleteThis/Scripts/../01_trimmed/test1.fq.gz.lane.clean.gz
Started analysis of test1.fq.gz.lane.clean.gz
Approx 10% complete for test1.fq.gz.lane.clean.gz
Approx 20% complete for test1.fq.gz.lane.clean.gz
Approx 30% complete for test1.fq.gz.lane.clean.gz
Approx 40% complete for test1.fq.gz.lane.clean.gz
Approx 50% complete for test1.fq.gz.lane.clean.gz
Approx 60% complete for test1.fq.gz.lane.clean.gz
Approx 75% complete for test1.fq.gz.lane.clean.gz
Approx 85% complete for test1.fq.gz.lane.clean.gz
Approx 95% complete for test1.fq.gz.lane.clean.gz
/Users/jrm/Desktop/deleteThis/Scripts/../01_trimmed/test2.fq.gz.lane.clean.gz
Started analysis of test2.fq.gz.lane.clean.gz
Approx 10% complete for test2.fq.gz.lane.clean.gz
Approx 20% complete for test2.fq.gz.lane.clean.gz
Approx 30% complete for test2.fq.gz.lane.clean.gz
Approx 40% complete for test2.fq.gz.lane.clean.gz
Approx 50% complete for test2.fq.gz.lane.clean.gz
Approx 60%

## Library Sizes

The `02a.GetLibrarySizes.sh`script will calculate the sizes (number of sequence reads) of raw and processed files; the  `02b.plotLibrarySizes.R` Rscript creates figures and tables.


In [70]:
Scripts/02a.GetLibrarySizes.sh


Started on Fri Oct  6 11:39:15 PDT 2017

Processing raw files
test1 10000
test2 10000
Processing trimmed files
test1.fq.gz 9291
test2.fq.gz 9496

 execution time was 0 s.

 Done Fri Oct  6 11:39:15 PDT 2017


In [75]:
Rscript Scripts/02b.plotLibrarySizes.R;
open images/Raw_and_Trimmed_LibrariesSize.pdf

null device 
          1 


<object data="images/Raw_and_Trimmed_LibrariesSize.pdf" type="application/pdf" width="300" height="200">
<a href="images/Raw_and_Trimmed_LibrariesSize.pdf">test.pdf</a>
</object>

## Alignment


### Build index

Run the `03a_makeBowtieIndex.sh -h` to see how the script works help. 

You need a fasta file with either genome or cDNA information of your favourite organism to use as a reference and select a version of bowtie (1 or 2). This depends on the length of the sequencing reads. Depending on the bowtie version and reference used the output folders will be named differently but they're contained within a `RefIdx` folder. 

For short reads < 50bp bowtie 1 is recommended; for longer reads, use bowtie 2. 

Valid examples to call the script are: 

> `Scripts/03a_makeBowtieIndex.sh -v bwt1 -g meta/genome_ath_TAIRv10.fa -c meta/cdna_ath_TAIRv10.fa` <br>
> `Scripts/03a_makeBowtieIndex.sh -v bwt1 -g meta/genome_ath_TAIRv10.fa` <br>
> `Scripts/03a_makeBowtieIndex.sh -v bwt1 -c meta/cdna_ath_TAIRv10.fa` <br>

__Attention__: If you run the script with the -s option and assign a suffix, you must use the same name in the alignment script to call the appropriate index when aligning trimmed reads. The default suffix is 'ref' which isn't very informative. 



(This might take some time...)

### Align the clean reads

Use `Scripts/03b_bowtieMapping.sh -h` to see the help. 

In [54]:
# Mapping to the genome
Scripts/03b_bowtieMapping.sh -v bwt1 -r genome -a meta/bow1Args.txt



bwParams -a --best --strata -n 1 -m 1 -p 4 --sam --tryhard


Started on Fri Oct  6 11:27:38 PDT 2017

Genome
running bow1
Align with bowtie 1 

test1.fq.gz
# reads processed: 9291
# reads with at least one reported alignment: 7122 (76.65%)
# reads that failed to align: 1846 (19.87%)
# reads with alignments suppressed due to -m: 323 (3.48%)
Reported 7122 alignments to 1 output stream(s)
test2.fq.gz
# reads processed: 9496
# reads with at least one reported alignment: 6616 (69.67%)
# reads that failed to align: 1702 (17.92%)
# reads with alignments suppressed due to -m: 1178 (12.41%)
Reported 6616 alignments to 1 output stream(s)

Parameters used: -a --best --strata -n 1 -m 1 -p 4 --sam --tryhard 

 execution time was 2 s.

 Done Fri Oct  6 11:27:40 PDT 2017


In [55]:
# Mapping to the transcriptome with default options
Scripts/03b_bowtieMapping.sh -v bwt1 -r cDNA 



Bowtie parameters are empty, using default parameters
bwParams -a --best --strata -n 1 -m 1 -p 4 --sam --tryhard


Started on Fri Oct  6 11:27:45 PDT 2017

cDNA
running bow1
Align with bowtie 1 

test1.fq.gz
# reads processed: 9291
# reads with at least one reported alignment: 7905 (85.08%)
# reads that failed to align: 1020 (10.98%)
# reads with alignments suppressed due to -m: 366 (3.94%)
Reported 7905 alignments to 1 output stream(s)
test2.fq.gz
# reads processed: 9496
# reads with at least one reported alignment: 7379 (77.71%)
# reads that failed to align: 1029 (10.84%)
# reads with alignments suppressed due to -m: 1088 (11.46%)
Reported 7379 alignments to 1 output stream(s)

Parameters used: -a --best --strata -n 1 -m 1 -p 4 --sam --tryhard --norc

 execution time was 3 s.

 Done Fri Oct  6 11:27:48 PDT 2017


### Mapping efficiency

Bowtie outputs the fraction of reads that were uniquely aligned to the reference (also the fraction of non-mapped and supressed). The `03c_BowtieAlignmentStats.R` Rscript 

Choose one option: `cDNAbwt1`,`genomebwt1`,`genomebwt2`,`cDNAbwt2`. In the future I plan to make this as an input argument when calling the script, for now, edit the script so the appropriate data set will be analyzed.

In [82]:
Rscript Scripts/03c_BowtieAlignmentStats.R; #With genomebwt1
open images/MappingStats_genomebwt1.pdf

[1] 2
[1] "logs/bowStats_genomebwt1/test1.fq.gz.log"
[2] "logs/bowStats_genomebwt1/test2.fq.gz.log"
Doing Stats for Bowtie 1[1] "logs/bowStats_genomebwt1/test1.fq.gz.log"
[1] "logs/bowStats_genomebwt1/test2.fq.gz.log"
null device 
          1 


In [84]:
Rscript Scripts/03c_BowtieAlignmentStats.R; #With genomebwt1
open images/MappingStats_cDNAbwt1.pdf

[1] 2
[1] "logs/bowStats_cDNAbwt1/test1.fq.gz.log"
[2] "logs/bowStats_cDNAbwt1/test2.fq.gz.log"
Doing Stats for Bowtie 1[1] "logs/bowStats_cDNAbwt1/test1.fq.gz.log"
[1] "logs/bowStats_cDNAbwt1/test2.fq.gz.log"
null device 
          1 


### Annotation and Counts

#### For genome alignments

Use `Scripts/04a_annotationHTSeq.sh -h` to see the help. 

#### For cDNA alignments

Use `Scripts/04b_countsFromcDNAMapping.sh -h` to see the help. 

In [56]:
Scripts/04a_annotationHTSeq.sh -v bwt1 -a meta/TAIR10_withTransposons.gff -p meta/htseqArgs.txt





Started on Fri Oct  6 11:27:53 PDT 2017

Processing test1.fq.gz.bam
100000 GFF lines processed.
200000 GFF lines processed.
300000 GFF lines processed.
400000 GFF lines processed.
500000 GFF lines processed.
600000 GFF lines processed.
656309 GFF lines processed.
7122 SAM alignments  processed.

Processing test2.fq.gz.bam
100000 GFF lines processed.
200000 GFF lines processed.
300000 GFF lines processed.
400000 GFF lines processed.
500000 GFF lines processed.
600000 GFF lines processed.
656309 GFF lines processed.
6616 SAM alignments  processed.







Parameters used: -f sam --stranded yes --mode intersection-strict --idattr ID --type gene meta/TAIR10_withTransposons.gff

 execution time was 28 s.

 Done Fri Oct  6 11:28:21 PDT 2017


In [60]:
Scripts/04b_countsFromcDNAMapping.sh -v bwt1



Started on Fri Oct  6 11:34:41 PDT 2017

Processing test1.fq.gz.bam

Processing test2.fq.gz.bam


Parameters used:  

 execution time was 0 s.

 Done Fri Oct  6 11:34:41 PDT 2017


In [62]:
head 04_Counts/bwt1_*/*

==> 04_Counts/bwt1_cDNA/test1.fq.gz.counts.txt <==
AT1G01560.2 1
AT1G01710.1 2
AT1G01720.1 3
AT1G01730.1 2
AT1G02080.1 1
AT1G02100.3 1
AT1G02220.1 8
AT1G02360.1 1
AT1G02500.1 1
AT1G02780.1 1

==> 04_Counts/bwt1_cDNA/test2.fq.gz.counts.txt <==
AT1G01120.1 1
AT1G01140.3 1
AT1G01360.1 1
AT1G01470.1 1
AT1G01550.1 2
AT1G01720.1 9
AT1G01820.1 1
AT1G01940.1 1
AT1G01960.1 1
AT1G02080.1 2

==> 04_Counts/bwt1_genome/htseqraw <==
head: Error reading 04_Counts/bwt1_genome/htseqraw

==> 04_Counts/bwt1_genome/test1.fq.gz.counts.txt <==
AT1G01010	0
AT1G01020	0
AT1G01030	0
AT1G01040	0
AT1G01046	0
AT1G01050	0
AT1G01060	0
AT1G01070	0
AT1G01073	0
AT1G01080	0

==> 04_Counts/bwt1_genome/test2.fq.gz.counts.txt <==
AT1G01010	0
AT1G01020	0
AT1G01030	0
AT1G01040	0
AT1G01046	0
AT1G01050	0
AT1G01060	0
AT1G01070	0
AT1G01073	0
AT1G01080	0


In [67]:
grep "AT1G01720" 04_Counts/bwt1_cDNA/test*
grep "AT1G01720" 04_Counts/bwt1_genome/test*

04_Counts/bwt1_cDNA/test1.fq.gz.counts.txt:AT1G01720.1 3
04_Counts/bwt1_cDNA/test2.fq.gz.counts.txt:AT1G01720.1 9
04_Counts/bwt1_genome/test1.fq.gz.counts.txt:AT1G01720	0
04_Counts/bwt1_genome/test2.fq.gz.counts.txt:AT1G01720	0


### Matrix of Counts

From either the genome or cDNA alignemnts and counts generated the `04c_makeItCount.R` script will create a matrix of counts. The script makes sure that the rows (gene names) are the same between files and that columns are in the correct order.

Two options can be selected in the Script. I plan to make it to accept arguments, but for the moment open the script and comment one of the following options:

> `option="bwt1_cDNA"` <br>
> `option="bwt1_genome"` <br>

In [79]:
Rscript Scripts/04c_makeItCount.R #With option="bwt1_genome"

Warning message:
In dir.create(imgPath) : 'images' already exists
[1] "test1.fq.gz.counts.txt" "test2.fq.gz.counts.txt"
-- make condenseListTables 
Filling table: test1.fq.gz.counts.txt 
Filling table: test2.fq.gz.counts.txt 
-- Condense table done 

          test1.fq.gz test2.fq.gz
AT1G01010           0           0
AT1G01020           0           0
AT1G01030           0           0
AT1G01040           0           0
AT1G01046           0           0
AT1G01050           0           0
[1] 28775     2
Plotting htseq stats 
Error in `colnames<-`(`*tmp*`, value = c("test1.fq.gz", "test2.fq.gz")) : 
  attempt to set 'colnames' on an object with less than two dimensions
Execution halted


In [77]:
Rscript Scripts/04c_makeItCount.R #With option="bwt1_cDNA"

Warning message:
In dir.create(imgPath) : 'images' already exists
[1] "test1.fq.gz.counts.txt" "test2.fq.gz.counts.txt"
-- make condenseListTables 
Filling table: test1.fq.gz.counts.txt 
Filling table: test2.fq.gz.counts.txt 
-- Condense table done 

            test1.fq.gz test2.fq.gz
AT1G01560.2           1           0
AT1G01710.1           2           0
AT1G01720.1           3           9
AT1G01730.1           2           0
AT1G02080.1           1           2
AT1G02100.3           1           0
[1] 2452    2
Skipping htseq stats 
[1] 2452    2
Using Library as id variables
null device 
          1 


This script also produces images:

In [78]:
open images/count_stats_bwt1_cDNA.pdf